In [15]:
# uncomment to install the required packages
#! pip install openmeteo-requests
#! pip install requests-cache retry-requests
#! pip install geopy

In [16]:
import pandas as pd
import geopy
import openmeteo_requests
import requests_cache
from retry_requests import retry
from dotenv import load_dotenv
import os

In [17]:
# Load the .env file
load_dotenv()

# Get the API key from environment variables
api_key = os.getenv("API_KEY")

## API usage: extract historical data

In [18]:
# Boilerplate code for the auto-generated Open-Meteo API
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [19]:
## Extract weather data from the hustorical weather API https://open-meteo.com/en/docs/historical-weather-api

In [20]:
url = "https://customer-archive-api.open-meteo.com/v1/archive" # url of the commercial API
params = {
	"latitude": 52.52, # you can also pass a list of coordinates to get data for multiple locations
	"longitude": 13.41,
	"start_date": "2024-06-01",
	"end_date": "2024-06-15",
	"hourly": "temperature_2m",
    "apikey": api_key
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m
0,2024-06-01 00:00:00+00:00,15.058500
1,2024-06-01 01:00:00+00:00,16.008499
2,2024-06-01 02:00:00+00:00,16.008499
3,2024-06-01 03:00:00+00:00,15.658501
4,2024-06-01 04:00:00+00:00,16.158501


In [21]:
# The API accepts list of longitude and latitude, so we can request multiple locations at once.
# We need to know latitude and longitude of locations we want to request.
# The task is called "geocoding" and it is a common task in data analysis.
# Geopy could be a good tool to do the job
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my_app')
loc = geolocator.geocode("Lugano, Switzerland")
loc, loc.longitude, loc.latitude

(Location(Lugano, Circolo di Lugano ovest, Distretto di Lugano, Ticino, Schweiz/Suisse/Svizzera/Svizra, (46.0050102, 8.9520281, 0.0)),
 8.9520281,
 46.0050102)

## API usage: extract climate model predictions

In [22]:
## Extract climate model predictions from the API https://open-meteo.com/en/docs/climate-api
# You can extract model predictions from different models, e.g. GFS, ICON, ECMWF, etc.
# and for different locations. You can compare these sophisticated models with your simple linear model.

In [23]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
#url = "https://climate-api.open-meteo.com/v1/climate"
url = "https://customer-climate-api.open-meteo.com/v1/climate"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "1950-01-01",
	"end_date": "2050-12-31",
	"models": ["CMCC_CM2_VHR4", "FGOALS_f3_H", "HiRAM_SIT_HR", "MRI_AGCM3_2_S", "EC_Earth3P_HR", "MPI_ESM1_2_XR", "NICAM16_8S"],
	"daily": "temperature_2m_mean",
    "apikey": "5ftjs2VK6XVIpyYM" # commercial API key, with more tokens available than the free API key
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.head()

Coordinates 52.5°N 13.400009155273438°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean
0,1950-01-01 00:00:00+00:00,-3.266310
1,1950-01-02 00:00:00+00:00,-1.464694
2,1950-01-03 00:00:00+00:00,-0.163077
3,1950-01-04 00:00:00+00:00,0.938540
4,1950-01-05 00:00:00+00:00,-4.809843
